In [11]:
import sys  
sys.path.insert(0, '../OMPbenders/')

In [12]:
import pandas as pd
import gurobipy   as      gp
from   gurobipy   import GRB
from MasterProblemOMP import MasterProblem
from SubProblemOMP import SubProblem

AL utilizar ql_t en 0.15 la solución objetivo se convierte en 0.0, estamos usando Ton * %Cu como ql, ¿Esta bien???

In [13]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'Modelo_F_OG_4_4_4.xlsx'

In [14]:
if undergroundDatabaseName == openPitDatabaseName:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = undergroundMineDataframe
else:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [15]:
class Main:
    def __init__(self, undergroundMineDataframe, openPitDataframe):
        self.openPitDataframe = openPitDataframe
        self.undergroundMineDataframe = undergroundMineDataframe
        self.numberOfPeriods = 5
        self.epsilon = 1000
        
    def execute(self):
        self.createModels()
        self.setMasterProblemCrownPillarHeights()
        self.getResults()
    
    def createModels(self):
        self.createSubProblem()
        self.createMasterProblem()

    def createSubProblem(self):
        self.SubProblem = SubProblem(self.openPitDataframe, self.numberOfPeriods)
        self.SubProblem.setParameters()

    def createMasterProblem(self):
        self.MasterProblem = MasterProblem(self.undergroundMineDataframe, self.numberOfPeriods)
        self.MasterProblem.setParameters()
        
    def setMasterProblemCrownPillarHeights(self):
        self.MasterProblem.V, self.MasterProblem.rho_v = self.SubProblem.V, self.SubProblem.rho_v
        self.MasterProblem.B_v = self.SubProblem.B_v

    def getResults(self):
        it = 0
        self.MasterProblem.setModel()
        currentTheta = -100000000000
        while True:
            it +=1
            self.estimatedW_v, self.estimatedTheta = self.MasterProblem.optimize()
            numberOfInvaiableBlocks = len(self.MasterProblem.B_v[self.MasterProblem.selectedHeight])
            subProblemObjValue, pi_vb = self.SubProblem.execute(numberOfInvaiableBlocks)
            if abs(self.MasterProblem.theta.X - currentTheta) <= self.epsilon:
                subProblemObjValue, pi_vb = self.SubProblem.execute(numberOfInvaiableBlocks, isFinalIteration = True)
                currentTheta = self.estimatedTheta.X
                self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= subProblemObjValue + gp.quicksum(gp.quicksum(pi_vb[t] * len(self.MasterProblem.B_v[v]) * (self.estimatedW_v[v] - self.MasterProblem.w_v[v]) for t in self.MasterProblem.t_S) for v in self.MasterProblem.V))       
                self.estimatedW_v, self.estimatedTheta = self.MasterProblem.optimize()
                print()
                print("-----------------------------Valor óptimo encontrado-----------------------------")
                print("Optimal Crown Pillar height: {}".format(self.estimatedW_v))
                print("Optimal ObjVal: {}".format(self.MasterProblem.undergroundModel.objVal))
                print("Optimal Theta: {}".format(self.MasterProblem.theta.X))
                #print("Current theta: {}".format(currentTheta))
                break
            
            print(it, "Objective Value =", self.MasterProblem.undergroundModel.objVal)
            print("w_v: {}".format(self.estimatedW_v))
            print("Master theta: {}".format(self.MasterProblem.theta))
            print("Current theta: {}".format(currentTheta))
            currentTheta = self.estimatedTheta.X
            self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= subProblemObjValue + gp.quicksum(gp.quicksum(pi_vb[t] * len(self.MasterProblem.B_v[v]) * (self.estimatedW_v[v] - self.MasterProblem.w_v[v]) for t in self.MasterProblem.t_S) for v in self.MasterProblem.V))
            print()
            print()

In [16]:
main = Main(undergroundMineDataframe, openPitDataframe)
main.execute()

Set parameter TimeLimit to value 3600
1 Objective Value = 800000000.0
w_v: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Master theta: <gurobi.Var theta (value 800000000.0)>
Current theta: -100000000000


2 Objective Value = 668580000.0
w_v: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Master theta: <gurobi.Var theta (value 668580000.0)>
Current theta: 800000000.0



-----------------------------Valor óptimo encontrado-----------------------------
Optimal Crown Pillar height: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Optimal ObjVal: -0.0
Optimal Theta: 0.0


In [17]:
main.estimatedW_v

{745: 1.0,
 785: 0.0,
 825: 0.0,
 865: 0.0,
 905: 0.0,
 945: 0.0,
 985: 0.0,
 1025: 0.0,
 1065: 0.0,
 1105: 0.0,
 1145: 0.0,
 1185: 0.0,
 1225: 0.0,
 1265: 0.0,
 1305: 0.0}

In [18]:
main.MasterProblem.selectedHeight

745

In [19]:
main.MasterProblem.undergroundModel.getAttr(GRB.Attr.X, main.MasterProblem.undergroundModel.getVars())

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [20]:
len(main.MasterProblem.undergroundBlocks)


1003520